In [1]:
# Bibliotecas
import sys
import os
from newsapi import NewsApiClient
import pandas as pd
import numpy as np

In [2]:
# Variáveis
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "env")))
from env_variables import apikey as apikey_temp

In [3]:
# Scripts
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "api")))
import api

sys.path.append(os.path.abspath(os.path.join(os.path.dirname("tests.ipynb"), "..", "pipeline")))
import pipeline

In [4]:
news_api = api.api_client(apikey_temp)
news_api

In [5]:
all_articles = api.get_articles(news_api)

In [6]:
len(all_articles)

500

In [7]:
df = pd.DataFrame(data=all_articles)
df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,"{'id': None, 'name': 'NPR'}",Nathan Rott,A “golden age” of rat research may be here. Wh...,"Rat and human lives have long intersected, but...",https://www.npr.org/sections/shots-health-news...,https://npr.brightspotcdn.com/dims3/default/st...,2024-09-20T11:23:58Z,When ecologist Jason Munshi-South started stud...
1,"{'id': None, 'name': 'BBC News'}",None,First newborns join screening for 200 rare dis...,The new study uses genome sequencing to detect...,https://www.bbc.com/news/articles/c70z8ppjlddo,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-10-03T00:05:37Z,Dominika's one-day-old daughter Emilia is part...
2,"{'id': None, 'name': 'Science Daily'}",None,Genomics reveals sled dogs' Siberian lineage,New research examines thousands of years of Ar...,https://www.sciencedaily.com/releases/2024/09/...,https://www.sciencedaily.com/images/scidaily-i...,2024-09-16T15:55:30Z,New research co-led by Cornell University exam...
3,"{'id': None, 'name': 'Science Daily'}",None,"Outbreak detection under-resourced in Asia, st...",A new study has revealed that despite the rece...,https://www.sciencedaily.com/releases/2024/09/...,https://www.sciencedaily.com/images/scidaily-i...,2024-09-24T16:30:08Z,A landmark study led by Duke-NUS Medical Schoo...
4,"{'id': 'business-insider', 'name': 'Business I...",Darius Rafieyan,OpenAI's investors include Silicon Valley's mo...,OpenAI's investors include Silicon Valley's mo...,https://www.businessinsider.com/openai-investo...,https://i.insider.com/65081a0cf05aa00019bdfa0f...,2024-10-03T13:00:01Z,SoftBank boss Masayoshi Son. Tomohiro Ohsumi/G...


In [8]:
df = pipeline.extract_font(df)

df.head()

,source,author,title,description,url,urlToImage,publishedAt,content
0,NPR,Nathan Rott,A “golden age” of rat research may be here. Wh...,"Rat and human lives have long intersected, but...",https://www.npr.org/sections/shots-health-news...,https://npr.brightspotcdn.com/dims3/default/st...,2024-09-20T11:23:58Z,When ecologist Jason Munshi-South started stud...
1,BBC News,None,First newborns join screening for 200 rare dis...,The new study uses genome sequencing to detect...,https://www.bbc.com/news/articles/c70z8ppjlddo,https://ichef.bbci.co.uk/news/1024/branded_new...,2024-10-03T00:05:37Z,Dominika's one-day-old daughter Emilia is part...
2,Science Daily,None,Genomics reveals sled dogs' Siberian lineage,New research examines thousands of years of Ar...,https://www.sciencedaily.com/releases/2024/09/...,https://www.sciencedaily.com/images/scidaily-i...,2024-09-16T15:55:30Z,New research co-led by Cornell University exam...
3,Science Daily,None,"Outbreak detection under-resourced in Asia, st...",A new study has revealed that despite the rece...,https://www.sciencedaily.com/releases/2024/09/...,https://www.sciencedaily.com/images/scidaily-i...,2024-09-24T16:30:08Z,A landmark study led by Duke-NUS Medical Schoo...
4,Business Insider,Darius Rafieyan,OpenAI's investors include Silicon Valley's mo...,OpenAI's investors include Silicon Valley's mo...,https://www.businessinsider.com/openai-investo...,https://i.insider.com/65081a0cf05aa00019bdfa0f...,2024-10-03T13:00:01Z,SoftBank boss Masayoshi Son. Tomohiro Ohsumi/G...


In [9]:
df.columns

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')

In [10]:
df.shape

(500, 8)

In [12]:
df = pipeline.verificar_e_excluir_duplicadas(df)



Total de linhas duplicadas: 1
             source           author  \
399  ETF Daily News  MarketBeat News   
405  ETF Daily News  MarketBeat News   

                                                 title  \
399  KPP Advisory Services LLC Grows Stake in Agile...   
405  KPP Advisory Services LLC Grows Stake in Agile...   

                                           description  \
399  KPP Advisory Services LLC grew its holdings in...   
405  KPP Advisory Services LLC grew its holdings in...   

                                                   url  \
399  https://www.etfdailynews.com/2024/10/02/kpp-ad...   
405  https://www.etfdailynews.com/2024/10/02/kpp-ad...   

                                            urlToImage           publishedAt  \
399  https://www.americanbankingnews.com/wp-content...  2024-10-02T14:06:47Z   
405  https://www.americanbankingnews.com/wp-content...  2024-10-02T14:06:47Z   

                                               content  
399  KPP Advisory Services

In [13]:

df.shape


(499, 8)

In [15]:
pipeline.valores_nulos(df)

source          0
author         45
title           1
description     1
url             0
urlToImage     26
publishedAt     0
content         0
dtype: int64


In [17]:
df = pipeline.remover_coluna(df)
df.head()

,source,author,title,description,url,publishedAt,content
0,NPR,Nathan Rott,A “golden age” of rat research may be here. Wh...,"Rat and human lives have long intersected, but...",https://www.npr.org/sections/shots-health-news...,2024-09-20T11:23:58Z,When ecologist Jason Munshi-South started stud...
1,BBC News,None,First newborns join screening for 200 rare dis...,The new study uses genome sequencing to detect...,https://www.bbc.com/news/articles/c70z8ppjlddo,2024-10-03T00:05:37Z,Dominika's one-day-old daughter Emilia is part...
2,Science Daily,None,Genomics reveals sled dogs' Siberian lineage,New research examines thousands of years of Ar...,https://www.sciencedaily.com/releases/2024/09/...,2024-09-16T15:55:30Z,New research co-led by Cornell University exam...
3,Science Daily,None,"Outbreak detection under-resourced in Asia, st...",A new study has revealed that despite the rece...,https://www.sciencedaily.com/releases/2024/09/...,2024-09-24T16:30:08Z,A landmark study led by Duke-NUS Medical Schoo...
4,Business Insider,Darius Rafieyan,OpenAI's investors include Silicon Valley's mo...,OpenAI's investors include Silicon Valley's mo...,https://www.businessinsider.com/openai-investo...,2024-10-03T13:00:01Z,SoftBank boss Masayoshi Son. Tomohiro Ohsumi/G...


In [19]:
df = pipeline.alterar_nome_colunas(df)

df.head()

,Fonte,Autor,Título,Descrição,URL,Data Publicação,Conteúdo
0,NPR,Nathan Rott,A “golden age” of rat research may be here. Wh...,"Rat and human lives have long intersected, but...",https://www.npr.org/sections/shots-health-news...,2024-09-20T11:23:58Z,When ecologist Jason Munshi-South started stud...
1,BBC News,None,First newborns join screening for 200 rare dis...,The new study uses genome sequencing to detect...,https://www.bbc.com/news/articles/c70z8ppjlddo,2024-10-03T00:05:37Z,Dominika's one-day-old daughter Emilia is part...
2,Science Daily,None,Genomics reveals sled dogs' Siberian lineage,New research examines thousands of years of Ar...,https://www.sciencedaily.com/releases/2024/09/...,2024-09-16T15:55:30Z,New research co-led by Cornell University exam...
3,Science Daily,None,"Outbreak detection under-resourced in Asia, st...",A new study has revealed that despite the rece...,https://www.sciencedaily.com/releases/2024/09/...,2024-09-24T16:30:08Z,A landmark study led by Duke-NUS Medical Schoo...
4,Business Insider,Darius Rafieyan,OpenAI's investors include Silicon Valley's mo...,OpenAI's investors include Silicon Valley's mo...,https://www.businessinsider.com/openai-investo...,2024-10-03T13:00:01Z,SoftBank boss Masayoshi Son. Tomohiro Ohsumi/G...


In [21]:
df = pipeline.organizar_colunas(df)

df.head()

,Data Publicação,Título,Autor,Descrição,URL,Fonte,Conteúdo
0,2024-09-20T11:23:58Z,A “golden age” of rat research may be here. Wh...,Nathan Rott,"Rat and human lives have long intersected, but...",https://www.npr.org/sections/shots-health-news...,NPR,When ecologist Jason Munshi-South started stud...
1,2024-10-03T00:05:37Z,First newborns join screening for 200 rare dis...,None,The new study uses genome sequencing to detect...,https://www.bbc.com/news/articles/c70z8ppjlddo,BBC News,Dominika's one-day-old daughter Emilia is part...
2,2024-09-16T15:55:30Z,Genomics reveals sled dogs' Siberian lineage,None,New research examines thousands of years of Ar...,https://www.sciencedaily.com/releases/2024/09/...,Science Daily,New research co-led by Cornell University exam...
3,2024-09-24T16:30:08Z,"Outbreak detection under-resourced in Asia, st...",None,A new study has revealed that despite the rece...,https://www.sciencedaily.com/releases/2024/09/...,Science Daily,A landmark study led by Duke-NUS Medical Schoo...
4,2024-10-03T13:00:01Z,OpenAI's investors include Silicon Valley's mo...,Darius Rafieyan,OpenAI's investors include Silicon Valley's mo...,https://www.businessinsider.com/openai-investo...,Business Insider,SoftBank boss Masayoshi Son. Tomohiro Ohsumi/G...


In [23]:
df = pipeline.data(df)

df.head()


,Data Publicação,Título,Autor,Descrição,URL,Fonte,Conteúdo
0,2024/09/20,A “golden age” of rat research may be here. Wh...,Nathan Rott,"Rat and human lives have long intersected, but...",https://www.npr.org/sections/shots-health-news...,NPR,When ecologist Jason Munshi-South started stud...
1,2024/10/03,First newborns join screening for 200 rare dis...,None,The new study uses genome sequencing to detect...,https://www.bbc.com/news/articles/c70z8ppjlddo,BBC News,Dominika's one-day-old daughter Emilia is part...
2,2024/09/16,Genomics reveals sled dogs' Siberian lineage,None,New research examines thousands of years of Ar...,https://www.sciencedaily.com/releases/2024/09/...,Science Daily,New research co-led by Cornell University exam...
3,2024/09/24,"Outbreak detection under-resourced in Asia, st...",None,A new study has revealed that despite the rece...,https://www.sciencedaily.com/releases/2024/09/...,Science Daily,A landmark study led by Duke-NUS Medical Schoo...
4,2024/10/03,OpenAI's investors include Silicon Valley's mo...,Darius Rafieyan,OpenAI's investors include Silicon Valley's mo...,https://www.businessinsider.com/openai-investo...,Business Insider,SoftBank boss Masayoshi Son. Tomohiro Ohsumi/G...


In [25]:
df = pipeline.ordenar_data(df)

df.head()

,Data Publicação,Título,Autor,Descrição,URL,Fonte,Conteúdo
0,2024/09/16,A new lineage nomenclature to aid genomic surv...,"Verity Hill, Sara Cleemput, James Siqueira Per...",Dengue virus (DENV) is causing increasingly la...,https://journals.plos.org/plosbiology/article?...,Plos.org,"Citation: Hill V, Cleemput S, Pereira JS, Giff..."
1,2024/09/16,Cell-Free DNA Isolation and Extraction Market ...,Research and Markets,"Dublin, Sept. 16, 2024 (GLOBE NEWSWIRE) -- The...",https://www.globenewswire.com/news-release/202...,GlobeNewswire,"Dublin, Sept. 16, 2024 (GLOBE NEWSWIRE) -- The..."
2,2024/09/16,Victory Capital Management Inc. Raises Stock H...,MarketBeat News,Victory Capital Management Inc. lifted its hol...,https://www.etfdailynews.com/2024/09/16/victor...,ETF Daily News,Victory Capital Management Inc. lifted its hol...
3,2024/09/16,Artificial Intelligence (AI) Radiology Tool Re...,Research and Markets,"Dublin, Sept. 16, 2024 (GLOBE NEWSWIRE) -- The...",https://www.globenewswire.com/news-release/202...,GlobeNewswire,"Dublin, Sept. 16, 2024 (GLOBE NEWSWIRE) -- The..."
4,2024/09/16,Stephen Fry – AI: A Means to an End or a Means...,Stephen Fry,The text of a talk I gave on Thursday 12th Sep...,https://stephenfry.substack.com/p/ai-a-means-t...,Substack.com,Thank you all so much.\r\nSo many questions. T...
